In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

In [2]:
# Download training data from open datasets.

#class AddGaussianNoise(object):
#    def __init__(self, mean=0., std=1.):
#        self.std = std
#        self.mean = mean
#        
#    def __call__(self, tensor):
#        return tensor + torch.randn(tensor.size()) * self.std + self.mean
#    
#    def __repr__(self):
#        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform = transforms.Compose([transforms.Resize((128, 128)),
                                    transforms.ToTensor(),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                    transforms.RandomVerticalFlip(),
                                    transforms.RandomRotation(45)]),
)

# Download test data from open datasets.
test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform = transforms.Compose([transforms.Resize((128, 128)), transforms.ToTensor(),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]),
)

# Download test data from open datasets.
val_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform = transforms.Compose([transforms.Resize((128, 128)), transforms.ToTensor(),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]),
)

In [3]:
batch_size = 32
num_classes = 102
learning_rate = 0.001
num_epochs = 200

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
val_dataloader = DataLoader(val_data, batch_size=batch_size)

In [4]:
import torch.nn.functional as F
import os

class Net(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.batch_layer1 = nn.BatchNorm2d(num_features=64) 
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.batch_layer2 = nn.BatchNorm2d(num_features=128)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        self.batch_layer3 = nn.BatchNorm2d(num_features=256)
        self.max_pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3)
        self.batch_layer4 = nn.BatchNorm2d(num_features=512)
        self.max_pool4 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer5 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3)
        self.batch_layer5 = nn.BatchNorm2d(num_features=1024)
        self.max_pool5 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(4096 , 128)
        self.relu1 = nn.PReLU()
        self.fc2 = nn.Linear(128, num_classes)
    

    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.batch_layer1(out)
        out = self.relu1(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer2(out)
        out = self.batch_layer2(out)
        out = self.relu1(out)
        out = self.max_pool2(out)

        out = self.conv_layer3(out)
        out = self.batch_layer3(out)
        out = self.relu1(out)
        out = self.max_pool3(out)

        out = self.conv_layer4(out)
        out = self.batch_layer4(out)
        out = self.relu1(out)
        out = self.max_pool4(out)
        
        out = self.conv_layer5(out)
        out = self.batch_layer5(out)
        out = self.relu1(out)
        out = self.max_pool5(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

net = Net(num_classes)

def save_network(network, epoch_label):
    save_filename = 'net_%s.pth' % epoch_label
    save_path = os.path.join(save_filename)
    torch.save(network.state_dict(), save_path)

def testValidationData(network):
    net.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        return (100 * correct / total)

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.00005)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
saveCounter = 0
minLoss = 10000
valAccurary = 0

# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_dataloader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)
      
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    if minLoss > loss and epoch % 5 == 0:
        tempAcc = testValidationData(net)
        if tempAcc > valAccurary:
            valAccurary = tempAcc
            saveCounter += 1
            save_network(net, saveCounter)
            print("SAVE COMPLETE NEW ACC:", valAccurary)


    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

SAVE COMPLETE NEW ACC: 6.666666666666667
Epoch [1/200], Loss: 4.3954
Epoch [2/200], Loss: 4.0212
Epoch [3/200], Loss: 3.7634
Epoch [4/200], Loss: 3.0516
Epoch [5/200], Loss: 3.2515
SAVE COMPLETE NEW ACC: 21.568627450980394
Epoch [6/200], Loss: 3.0333
Epoch [7/200], Loss: 2.3861
Epoch [8/200], Loss: 2.2211
Epoch [9/200], Loss: 2.1164
Epoch [10/200], Loss: 1.7961
SAVE COMPLETE NEW ACC: 28.529411764705884
Epoch [11/200], Loss: 1.9183
Epoch [12/200], Loss: 1.7241
Epoch [13/200], Loss: 2.1033
Epoch [14/200], Loss: 1.5586
Epoch [15/200], Loss: 1.9532
SAVE COMPLETE NEW ACC: 36.86274509803921
Epoch [16/200], Loss: 1.6447
Epoch [17/200], Loss: 1.1169
Epoch [18/200], Loss: 1.5359
Epoch [19/200], Loss: 1.5785
Epoch [20/200], Loss: 1.1687
SAVE COMPLETE NEW ACC: 39.21568627450981
Epoch [21/200], Loss: 1.1692
Epoch [22/200], Loss: 1.2330
Epoch [23/200], Loss: 1.1217
Epoch [24/200], Loss: 1.0321
Epoch [25/200], Loss: 0.5985
SAVE COMPLETE NEW ACC: 44.6078431372549
Epoch [26/200], Loss: 1.4172
Epoch [2

In [7]:
net.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(total, 100 * correct / total))

Accuracy of the network on the 6149 train images: 47.145877378435515 %


In [10]:
save_network(net, "Final")